In [ ]:
# Clone code
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI.git ./ComfyUI
!git clone https://github.com/city96/ComfyUI-GGUF.git ./ComfyUI/custom_nodes/quantized

# Download library
%cd /content/ComfyUI
!pip install -q einops==0.8.0 torchsde==0.2.6 diffusers==0.31.0 accelerate==1.1.0 gguf==0.10.0 gradio==5.9.1
!pip install -q huggingface_hub==0.27.0 hf_transfer==0.1.8

# Download model
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download martintomov/Hyper-FLUX.1-dev-gguf hyper-flux-16step-Q4_K_M.gguf --local-dir ./models/unet --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download Comfy-Org/sigclip_vision_384 sigclip_vision_patch14_384.safetensors --local-dir ./models/clip_vision --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download Thelocallab/Flux-Dev-Redux flux1-redux-dev.safetensors --local-dir ./models/style_models --local-dir-use-symlinks False # originally, it's from black-forest-labs/FLUX.1-Redux-dev. Check the LICENSE.
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download camenduru/FLUX.1-dev ae.sft --local-dir ./models/vae --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download zer0int/CLIP-GmP-ViT-L-14 ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors --local-dir ./models/clip --local-dir-use-symlinks False
!HF_HUB_ENABLE_HF_TRANSFER=1 huggingface-cli download comfyanonymous/flux_text_encoders t5xxl_fp8_e4m3fn.safetensors --local-dir ./models/clip --local-dir-use-symlinks False

In [ ]:
# Set nodes
%cd /content/ComfyUI

import torch
import nodes
from nodes import NODE_CLASS_MAPPINGS
from comfy import model_management
from comfy_extras import nodes_custom_sampler, nodes_flux, nodes_model_advanced

CLIPVisionLoader = nodes.NODE_CLASS_MAPPINGS["CLIPVisionLoader"]()
LoadImage = nodes.NODE_CLASS_MAPPINGS["LoadImage"]()
StyleModelLoader = nodes.NODE_CLASS_MAPPINGS["StyleModelLoader"]()
CLIPVisionEncode = nodes.NODE_CLASS_MAPPINGS["CLIPVisionEncode"]()
StyleModelApply = nodes.NODE_CLASS_MAPPINGS["StyleModelApply"]()

DualCLIPLoader = nodes.NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
VAELoader = nodes.NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = nodes.NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
ModelSamplingFlux = nodes_model_advanced.NODE_CLASS_MAPPINGS["ModelSamplingFlux"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()

RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
EmptyLatentImage = nodes.NODE_CLASS_MAPPINGS["EmptyLatentImage"]()
VAEDecode = nodes.NODE_CLASS_MAPPINGS["VAEDecode"]()

# Import model
with torch.inference_mode():
  clip = DualCLIPLoader.load_clip("t5xxl_fp8_e4m3fn.safetensors", "ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors", "flux")[0]
  vae = VAELoader.load_vae("ae.sft")[0]
  clip_vision = CLIPVisionLoader.load_clip("sigclip_vision_patch14_384.safetensors")[0]
  style_model = StyleModelLoader.load_style_model("flux1-redux-dev.safetensors")[0]

from custom_nodes.quantized.nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()

with torch.inference_mode():
  unet = UNETLoader.load_unet("hyper-flux-16step-Q4_K_M.gguf")[0]

In [ ]:
import os
import random
import torch
import numpy as np
from PIL import Image
import gradio as gr

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

# Function to generate image
def generate_image(image, prompt, width, height, seed):
  with torch.inference_mode():
    positive_prompt = prompt
    width = width
    height = height
    seed = seed
    steps = 16
    sampler_name = "euler"
    scheduler = "simple"

    image.save("/content/ComfyUI/input/image.png")
    image = LoadImage.load_image("image.png")[0]
    clip_vision_output = CLIPVisionEncode.encode(clip_vision=clip_vision, image=image, crop="none")[0]

    cond = CLIPTextEncode.encode(text=prompt, clip=clip)[0]
    cond_f = FluxGuidance.append(conditioning=cond, guidance=3.5)[0]
    cond_ff = StyleModelApply.apply_stylemodel(conditioning=cond_f, style_model=style_model, clip_vision_output=clip_vision_output, strength=1.0, strength_type="multiply")[0]

    noise = RandomNoise.get_noise(seed)[0]
    new_unet = ModelSamplingFlux.patch(model=unet, width=width, height=height, max_shift=1.15, base_shift=0.5)[0]

    sampler = KSamplerSelect.get_sampler(sampler_name=sampler_name)[0]
    guider = BasicGuider.get_guider(model=new_unet, conditioning=cond_ff)[0]
    sigmas = BasicScheduler.get_sigmas(model=new_unet, scheduler=scheduler, steps=steps, denoise=1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]

    sample, _ = SamplerCustomAdvanced.sample(noise=noise, guider=guider, sampler=sampler, sigmas=sigmas, latent_image=latent_image)
    model_management.soft_empty_cache()

    decoded = VAEDecode.decode(vae=vae, samples=sample)[0].detach()
    image = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])
    return image

# Function to handle button click
def on_generate_click(image, prompt, width, height, seed):
    if not prompt.strip():
        return None, "❌ 텍스트를 입력해주세요.", None
    if not image:
        return None, "❌ 이미지를 입력해주세요.", None
    try:
        image = generate_image(image, prompt, width, height, seed)
        return image, "✅ 성공적으로 이미지를 생성했습니다.", seed
    except Exception as e:
        return None, f"❌ 에러 발생: {str(e)}", None

# Function to generate a random seed
def roll_seed():
    return random.randint(0, 2**32 - 1)

In [ ]:
# Create Gradio interface
def create_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# 🖼️ 플럭스 활용하기")
        with gr.Row():
            # Left column: Inputs
            with gr.Column(scale=1):
                image = gr.Image(
                    label="🖼️ 이미지",
                    type="pil",
                    height=300,
                )
                prompt = gr.Textbox(
                    label="📝 텍스트",
                    placeholder="원하는 장면을 영어로 입력해주세요...",
                    lines=3
                )
                width = gr.Slider(
                    label="📏 가로 (pixels)",
                    minimum=64,
                    maximum=1280,
                    step=64,
                    value=1024
                )
                height = gr.Slider(
                    label="📏 세로 (pixels)",
                    minimum=64,
                    maximum=1280,
                    step=64,
                    value=1024
                )
                with gr.Row():
                    seed = gr.Number(
                        label="🎲 시드",
                        value=2024,
                        precision=0,
                        interactive=True
                    )
                    roll_seed_btn = gr.Button("🎲 랜덤 시드 만들기")
                generate_btn = gr.Button("🎨 이미지 생성하기")
                progress = gr.Textbox(label="🔄 현재 상태", interactive=False)

            # Right column: Output
            with gr.Column(scale=1):
                output = gr.Image(label="🖼️ 생성한 이미지")
                used_seed = gr.Number(label="📌 사용된 시드", value=42, precision=0, interactive=False)

        # Define the button click events
        generate_btn.click(
            fn=on_generate_click,
            inputs=[image, prompt, width, height, seed],
            outputs=[output, progress, used_seed]
        )

        roll_seed_btn.click(
            fn=roll_seed,
            inputs=None,
            outputs=seed
        )

    return demo

if __name__ == "__main__":
    interface = create_interface()
    interface.launch()